Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from pymystem3 import Mystem
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('always')  

In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
comments.head(40)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [4]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
# переводим все в нижний регистр
comments['text'] = comments['text'].str.lower()

In [6]:
# делаем стемминг, т.к. язык английский
ps = PorterStemmer()
comments['lemm_text'] = comments['text'].apply(lambda x: [ps.stem(y) for y in x])

In [7]:
#m = Mystem()
#def lemmatize(text):
    #m = Mystem()
    #lemm_list = m.lemmatize(text)
    #lemm_text = ''.join(lemm_list)
    #return lemm_text

def clear_text(text): # чистим текст от лишних символов
    cleaned_text = re.sub(r'[^a-zA-Z]', ' ', text)
    return ' '.join(cleaned_text.split())

In [8]:
comments['lemm_text'] = comments['text'].apply(clear_text)

In [9]:
comments.head()

,text,toxic,lemm_text
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,d'aww! he matches this background colour i'm s...,0,d aww he matches this background colour i m se...
2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not trying to edit war it s...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...


In [10]:
print(comments['toxic'].value_counts())

0    143346
1     16225
Name: toxic, dtype: int64


Наблюдаем значительный дисбаланс классов.

In [11]:
train, valid = train_test_split(comments, random_state=12345, test_size=0.4) # делим на выборки в соотношении 3:1:1.
valid, test = train_test_split(valid, random_state=12345, test_size=0.5)

In [12]:
target_train = train['toxic']
target_valid = valid['toxic']
target_test = test['toxic']

In [13]:
corpus_train = train['lemm_text'].values.astype('U')
corpus_valid = valid['lemm_text'].values.astype('U')
corpus_test = test['lemm_text'].values.astype('U')

In [ ]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_valid = count_tf_idf.transform(corpus_valid)
tf_idf_test = count_tf_idf.transform(corpus_test)

# 2. Обучение

In [16]:
# Обучаем логистическую регрессию. Если указать class_weight='balanced', больший вес будет у редкого класса
model = LogisticRegression(class_weight='balanced')
model.fit(tf_idf_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
print('Валидационная выборка ', '', f1_score(target_valid, model.predict(tf_idf_valid)))
print('Тестовая выборка ', '', f1_score(target_test, model.predict(tf_idf_test)))

Валидационная выборка   0.7533644603131009
Тестовая выборка   0.7524587893059981


In [18]:
model = DecisionTreeClassifier(class_weight='balanced')
model.fit(tf_idf_train, target_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [19]:
print('Валидационная выборка ', '', f1_score(target_valid, model.predict(tf_idf_valid)))
print('Тестовая выборка ', '', f1_score(target_test, model.predict(tf_idf_test)))

Валидационная выборка   0.6401421930544162
Тестовая выборка   0.6366685299049749


In [20]:
model = RandomForestClassifier(class_weight='balanced')
model.fit(tf_idf_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [21]:
print('Валидационная выборка ', '', f1_score(target_valid, model.predict(tf_idf_valid)))
print('Тестовая выборка ', '', f1_score(target_test, model.predict(tf_idf_test)))

Валидационная выборка   0.6108374384236454
Тестовая выборка   0.6082624039858834


Лучший результат F1 имеем для модели логистической регрессии:
    - Валидационная выборка   0.7533644603131009
    - Тестовая выборка   0.7524587893059981

# 3. Выводы

В результате проделанной работы по подготовке данных из набор датасета с разметкой о токсичности правок:
    - перевод в нижний регистр
    - стемминг (английский язык)
    - очиствка от лишних символов.

Т.к. у датасета значительный дисбаланс классов (почти 10:1), при обучении моделей указывался class_weight='balanced'(больший вес будет у редкого класса).

При обучении модели решающего дерева получены следующие значения F1:
    - Валидационная выборка   0.6401421930544162
    - Тестовая выборка   0.6366685299049749
    
При обучении модели случайного леса получены следующие значения F1:
    - Валидационная выборка   0.6108374384236454
    - Тестовая выборка   0.6082624039858834

Лучший результат F1 имеем для модели логистической регрессии (последующий downsample и uppsampling эффекта не дал):
    - Валидационная выборка   0.7533644603131009
    - Тестовая выборка   0.7524587893059981
